In [ ]:
import os
from app_decomposer.utils_demo import *
from pprint import pprint

In [ ]:
ts = get_job_timeseries_from_file()
#ts = ts.pop("accessPattern")
pprint(ts["volume"])
#ts = ts.pop('processCount')

In [ ]:
job1 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607510, 1581607515, 1581607520, 1581607525])}

job2 = {'bytesRead': np.array([       2457,           8000,  0,  0]),
        'bytesWritten': np.array([         15404800,  91795200,   0,          0]),
        'timestamp': np.array([1581607520, 1581607525, 1581607530, 1581607535])}

job3 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607530, 1581607535, 1581607540, 1581607545])}

In [ ]:

jobid=3918
dataset_path = os.path.join(dirname(dirname(os.getcwd())), "dataset_generation", "dataset_generation")
job_files, job_ids, dataset_names = list_jobs(dataset_path)
# create two lists of job_ids where only few elements are differents


print(dataset_path)
print(job_ids)
print(job_files)
csv_file = job_files[job_ids.index("52508")]
df = pd.read_csv(csv_file, index_col=0)  # read the csv file into a dataframe
# print the columns of the dataframe to check if 'operationRead' exists
print(csv_file)
get_job_timeseries_from_file(job_id=7910)


In [1]:
import sys
sys.path.append("../")
import os
import unittest
from shutil import rmtree
import urllib3
from loguru import logger
from pydantic import BaseModel
from typing import List
from app_decomposer.connector import Connector, APIConnector, ApiModel
from app_decomposer.config_parser import Configuration
import unittest
import numpy as np
import pandas as pd
from unittest.mock import MagicMock
from app_decomposer.central_job import WorkflowSynthesizer, CentralJob, WorkflowSearcher, display_features_3d

CONFIG_FILE = os.path.join(os.path.dirname(os.getcwd()), "tests_integration",
                      "test_data", "test_docker_config.yaml")

In [6]:
config = Configuration(path=CONFIG_FILE)
keycloak_token = config.get_kc_token_docker()
connector = APIConnector(api_uri="http://localhost/pybackend/",
                              api_token=f"Bearer {keycloak_token}")

connector.check_connection()

False

In [7]:
searcher = WorkflowSearcher(connector)
df = searcher.search_workflows("Cryo_EM")
df.head()

2023-07-13 17:27:35.193 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending POST request to endpoint /ioi/workflows/


,id,instanceNumber,name,username,version,jobs,workflowStartTime,workflowEndTime,workflowDuration,accumulatedJobDuration
0,642b5fd8547fb888bac488ed,03042023,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 343407}],1680564180,1680564780,600,600
1,633ab5ebac213a8b8529d186,03102022,Cryo_EM,furmanek1,4.0.5-Bull.9,"[{'jobid': 273857}, {'jobid': 273858}, {'jobid...",1664792040,1665143626,351586,1520
2,634019e6ac213a8b852f5f9c,07102022_02,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277150}],1665145315,1665145367,52,52
3,63402075ac213a8b852f6821,07102022_large,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277155}],1665146995,1665150606,3611,3611
4,634015a2ac213a8b852f4ec5,071022_01,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277142}],1665144225,1665144787,562,562


In [11]:
# Extract workflows data
workflows = [searcher.extract_workflow_data(workflow_id) for workflow_id in df['id']]
print(workflows)


2023-07-13 17:32:47.674 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/642b5fd8547fb888bac488ed
2023-07-13 17:32:47.910 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/633ab5ebac213a8b8529d186
2023-07-13 17:32:48.170 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/634019e6ac213a8b852f5f9c
2023-07-13 17:32:48.411 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/63402075ac213a8b852f6821
2023-07-13 17:32:48.762 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/634015a2ac213a8b852f4ec5
2023-07-13 17:32:48.993 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/63e52bfdac213a8b85e98c21
2023-07-13 17:32

[{'bytesRead': [0, 590611, 1066545, 521120, 31808, 0, 162914, 3670661, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'bytesWritten': [0, 0, 0, 0, 0, 0, 0, 5719, 0, 0, 0, 0, 0, 593, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0, 0, 589, 0, 0, 0, 0], 'timestamp': [1680564180, 1680564185, 1680564190, 1680564195, 1680564200, 1680564205, 1680564210, 1680564215, 1680564220, 1680564225, 1680564230, 1680564235, 1680564240, 1680564245, 1680564250, 1680564255, 1

In [9]:
# Convert each workflow data into the format expected by CentralJob
central_jobs = []
for workflow in workflows:
    job = {
        'bytesRead': np.array(workflow['bytesRead']),
        'bytesWritten': np.array(workflow['bytesWritten'])
    }
    central_jobs.append(job)

# Use the CentralJob class to find the central job index for each workflow
central_job = CentralJob(central_jobs)
central_job_index = central_job.find_central_job()

# Print the index of the central job
print('Central job index:', central_job_index)
# Extract corresponding line from df
central_workflow = df.iloc[central_job_index]
print(central_workflow)

2023-07-13 17:28:02.286 | INFO     | app_decomposer.central_job:__init__:135 - Initializing CentralJob instance
2023-07-13 17:28:02.286 | INFO     | app_decomposer.central_job:find_central_job:208 - Finding central job
2023-07-13 17:28:02.287 | INFO     | app_decomposer.central_job:process:173 - Processing jobs data
2023-07-13 17:28:02.288 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.290 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.291 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.292 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.292 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.294 | INFO     | app_decomposer.central_job:fft_features:155 - Calculating FFT features
2023-07-13 17:28:02.295 | INFO    

Central job index: 6
id                        63be91efac213a8b85bc56eb
instanceNumber                            11012023
name                                       Cryo_EM
username                                 furmanek1
version                               4.0.5-Bull.9
jobs                           [{'jobid': 303176}]
workflowStartTime                       1673433580
workflowEndTime                         1673434196
workflowDuration                               616
accumulatedJobDuration                         616
Name: 6, dtype: object


In [10]:
central_job.features

array([[0.00000000e+00, 1.74722042e-03, 6.90117403e-04, 1.22182011e-03,
        9.17984172e-04, 3.87520379e-04, 1.41888750e-03, 2.59432242e-03,
        2.73115185e-03, 1.35214566e-03, 1.27468344e-03, 1.08596135e-03,
        9.26759687e-04, 9.18600559e-04, 9.26759687e-04, 1.08596135e-03,
        1.27468344e-03, 1.35214566e-03, 2.73115185e-03, 2.59432242e-03,
        1.41888750e-03, 3.87520379e-04, 9.17984172e-04, 0.00000000e+00,
        3.34068265e-06, 3.38310344e-06, 1.50471047e-06, 1.11311796e-06,
        1.22183567e-06, 1.51111794e-06, 1.36393391e-06, 1.51981440e-06,
        1.58746052e-06, 2.02905118e-06, 1.85377997e-06, 1.83849133e-06,
        2.56306549e-06, 1.83849133e-06, 1.85377997e-06, 2.02905118e-06,
        1.58746052e-06, 1.51981440e-06, 1.36393391e-06, 1.51111794e-06,
        1.22183567e-06, 1.11311796e-06],
       [0.00000000e+00, 9.62097050e-01, 1.54479091e-01, 1.18734703e-03,
        1.25756800e-03, 1.50719295e-03, 2.10362927e-03, 3.02392948e-03,
        4.42477567e-03,

In [7]:
fig = display_features_3d(central_job.features)
fig.show()

In [13]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
import plotly.graph_objects as go
import numpy as np

# Apply PCA to reduce the features to 3 dimensions
pca = PCA(n_components=3)
reduced_features = pca.fit_transform(central_job.features)

# Calculate the amount of variance captured by each principal component
explained_variance = pca.explained_variance_ratio_

# Calculate the total amount of variance captured
total_variance = np.sum(explained_variance)

print(f"Variance captured by each principal component: {explained_variance}")
print(f"Total variance captured: {total_variance}")
# Calculate the Euclidean distance from each job to the centroid
distances = euclidean_distances(reduced_features, [centroid])

# Find the index of the job that is closest to the centroid
closest_job_idx = np.argmin(distances)

# Create the figure
fig = go.Figure()

# For each set of reduced features, create a scatter3d trace and add it to the figure
for i, feature in enumerate(reduced_features):
    # Choose different color and size for the closest job
    if i == closest_job_idx:
        color = 'red'
        size = 15  # Bigger size for the closest job
        name = f'Job {i} (Closest to Centroid)'
    else:
        color = i
        size = 12  # Normal size for other jobs
        name = f'Job {i}'

    fig.add_trace(go.Scatter3d(
        x=[feature[0]],
        y=[feature[1]],
        z=[feature[2]],
        mode='markers',
        marker=dict(
            size=size,
            color=color,                
            colorscale='Viridis',   
            opacity=0.8,
        ),
        text=[f'Job {i}'],
        name=name
    ))

# Add the centroid to the plot
fig.add_trace(go.Scatter3d(
    x=[centroid[0]],
    y=[centroid[1]],
    z=[centroid[2]],
    mode='markers',
    marker=dict(
        size=12,
        color='black',   # set color to black
        opacity=0.8,
    ),
    text=['Centroid'],
    name='Centroid'
))

# Set the layout properties
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='PC1'),
        yaxis=dict(title='PC2'),
        zaxis=dict(title='PC3')
    )
)

# Show the plot
fig.show()
